In [1]:
def intersection(int1, int2):
    
    if (int1[0] <= int2[1] and int1[1] >= int2[0]): 
        return 'overlap'
    
    else:
        return 'not overlap'

    #case 1: partial intersection over the left or right border
    #if (int2[0]<=int1[0] and int2[1]>=int1[1]) or (int2[0]>=int1[0] and int2[1]<=int1[1]):
        #return min(int1[1],int2[1]) - max(int1[0],int2[0])
        #return 'overlap'

    #case 2: complete overlap of one interval by the other
    #elif (int2[0]>=int1[0] and int2[1]<=int1[1]) or (int2[0]<=int1[0] and int2[1]>=int1[1]):
        #return min (int2[1]-int2[0] , int1[1]-int1[0])
        #return 'overlap'

    #case 3: no overlap at all
    #else:
        #return 'not overlap'

In [7]:
import ROOT
import scipy
import numpy as np
#import statsmodels
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.proportion import proportions_ztest

In [17]:
type = 'udsg'

file_LLR = ROOT.TFile('bTagEfficiencies_94X_2018WPs_Fall17MC_NEW.root')
h_num_LLR = file_LLR.Get('h2_BTaggingEff_Num_'+type +'_M_ALL')
h_denom_LLR = file_LLR.Get('h2_BTaggingEff_Denom_'+type +'_M_ALL')

file_PI = ROOT.TFile('BTagEfficiency_deep_csv_pu_id_full.root')
h_num_PI = file_PI.Get('All/Efficiency/h2_Num_'+type +'_M_all')
h_denom_PI = file_PI.Get('All/Efficiency/h2_Denom_'+type +'_all_all')

In [18]:
#canvas = ROOT.TCanvas('', '', 600, 600)
#h_num_LLR.Draw('colz')
#canvas.Draw()

In [19]:
#canvas1 = ROOT.TCanvas('pippo', 'pippo', 600, 600)
#h_denom_PI.Draw('colz')
#canvas1.Draw()

In [20]:


print('{:10} {:10} {:10} {:10} {:10} {:10} {:10} {:15} {:15} {:10} {:10} {:10} {:15} {:15} {:10}'.
      format('pt_min', 'pt_max', 'eta_min', 'eta_max' , 'eff_PI', 'ci_PI_low', 'ci_PI_up', 'err_low_PI[%]', 'err_up_PI[%]', 'eff_LLR' , 'ci_LLR_low', 'ci_LLR_up','err_low_LLR[%]', 'err_up_LLR[%]', 'p_value'))

for bin_x in range(1, h_num_LLR.GetXaxis().GetNbins() + 1):
    for bin_y in range(1, h_num_LLR.GetYaxis().GetNbins() + 1):
        eff_PI = h_num_PI.GetBinContent(bin_x, bin_y)/h_denom_PI.GetBinContent(bin_x, bin_y)
        eff_LLR = h_num_LLR.GetBinContent(bin_x, bin_y)/h_denom_LLR.GetBinContent(bin_x, bin_y)
        ci_LLR = proportion_confint(count=h_num_LLR.GetBinContent(bin_x, bin_y), nobs=h_denom_LLR.GetBinContent(bin_x, bin_y), alpha=0.32, method='beta')
        ci_PI = proportion_confint(count=h_num_PI.GetBinContent(bin_x, bin_y), nobs=h_denom_PI.GetBinContent(bin_x, bin_y), alpha=0.32, method='beta')
        
        if(eff_PI==0):
            err_low_PI = 0
            err_up_PI = 0
        else :
            err_low_PI = ((eff_PI - ci_PI[0])/eff_PI)*100
            err_up_PI = ((ci_PI[1] - eff_PI)/eff_PI)*100
        
        if(eff_LLR==0):
            err_low_LLR = 0
            err_up_LLR = 0
        else: 
            err_low_LLR = ((eff_LLR - ci_LLR[0])/eff_LLR)*100
            err_up_LLR = ((ci_LLR[1] - eff_LLR)/eff_LLR)*100
        
        int2 = [ci_PI[0],ci_PI[1]]
        int1 = [ci_LLR[0],ci_LLR[1]]
        
        counts = np.array([h_num_PI.GetBinContent(bin_x, bin_y), h_num_LLR.GetBinContent(bin_x, bin_y)])
        nobs = np.array([h_denom_PI.GetBinContent(bin_x, bin_y), h_denom_LLR.GetBinContent(bin_x, bin_y)])
        stat, pval = proportions_ztest(counts, nobs)
        
    
        print('{:<10.2f} {:<10.2f} {:<10.2f} {:<10.2f} {:<10.4f} {:<10.4f} {:<10.4f} {:<15.2f} {:<15.2f} {:<10.4f} {:<10.4f} {:<10.4f} {:<15.2f} {:<15.2f} {:<10.3f}'.
              format(h_num_LLR.GetXaxis().GetBinLowEdge(bin_x), h_num_LLR.GetXaxis().GetBinLowEdge(bin_x+1),
                    h_num_LLR.GetYaxis().GetBinLowEdge(bin_y), h_num_LLR.GetYaxis().GetBinLowEdge(bin_y+1),
                    eff_PI, ci_PI[0], ci_PI[1], err_low_PI, err_up_PI, eff_LLR, ci_LLR[0], ci_LLR[1], err_low_LLR, err_up_LLR, pval))
        
        

        


pt_min     pt_max     eta_min    eta_max    eff_PI     ci_PI_low  ci_PI_up   err_low_PI[%]   err_up_PI[%]    eff_LLR    ci_LLR_low ci_LLR_up  err_low_LLR[%]  err_up_LLR[%]   p_value   
20.00      30.00      0.00       0.60       0.0065     0.0064     0.0067     1.80            1.83            0.0054     0.0050     0.0057     6.30            6.71            0.002     
20.00      30.00      0.60       1.20       0.0097     0.0095     0.0098     1.52            1.54            0.0086     0.0082     0.0091     5.08            5.34            0.035     
20.00      30.00      1.20       2.10       0.0149     0.0147     0.0151     1.11            1.13            0.0134     0.0129     0.0139     3.68            3.81            0.006     
20.00      30.00      2.10       2.40       0.0124     0.0120     0.0127     2.60            2.67            0.0130     0.0120     0.0141     7.99            8.65            0.549     
30.00      40.00      0.00       0.60       0.0052     0.0051     0.0053   